# Predict test dataset

In [11]:
import pickle
import pandas as pd
import numpy as np

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#ML Algoirthm
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
import sklearn.linear_model as linear_model
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor

In [2]:
df = pd.read_csv('./preprossed_data/preprossed_test.csv')

In [3]:
# load
with open('./models/lgb_model_full_data.pkl', 'rb') as f:
    lgb_model_full_data = pickle.load(f)

In [4]:
# load
with open('./models/ridge_model_full_data.pkl', 'rb') as f:
    ridge_model_full_data = pickle.load(f)

In [5]:
# load
with open('./models/svr_model_full_data.pkl', 'rb') as f:
    svr_model_full_data = pickle.load(f)

In [6]:
# load
with open('./models/gbr_model_full_data.pkl', 'rb') as f:
    gbr_model_full_data = pickle.load(f)

In [7]:
# load
with open('./models/xgb_model_full_data.pkl', 'rb') as f:
    xgb_model_full_data = pickle.load(f)

[18:23:49] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [8]:
# load
with open('./models/stack_reg_model.pkl', 'rb') as f:
    stack_reg_model = pickle.load(f)

[18:23:50] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:23:50] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:23:50] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [9]:
def blended_predictions(X, weight):
    return ((weight[0] * ridge_model_full_data.predict(X)) + \
            (weight[1] * svr_model_full_data.predict(X)) + \
            (weight[2] * gbr_model_full_data.predict(X)) + \
            (weight[3] * xgb_model_full_data.predict(X)) + \
            (weight[4] * lgb_model_full_data.predict(X)) + \
            (weight[5] * stack_reg_model.predict(np.array(X))))

In [12]:
# Blended model predictions
blended_predictions(df,[0.10,0.10,0.20,0.15,0.15,0.3])

array([8.88462283, 9.02380581, 9.13291909, ..., 9.06377246, 8.86091333,
       9.18353114])

In [14]:
# Read submission csv
submission = pd.read_csv("./raw_data/sample_submission.csv")
# Predictions
submission.iloc[:,1] = np.floor(np.expm1(blended_predictions(df,[0.15,0.2,0.1,0.15,0.1,0.3])))

In [16]:
# Write to csv
submission.to_csv("./submission/submission.csv", index=False)